In [1]:
import numpy as np
import pandas as pd
import fasttext


In [4]:
df_sample = pd.read_csv('datasets/random_sample_data.csv')


In [5]:
# remove unnecessary comments
df_sample = df_sample.drop([x for x in range(len(df_sample)) if int(
    df_sample['label'][x]) != 0 and int(df_sample['label'][x]) != 1])

df_sample.tail()

,raw_comment,tokenized_comment,label
3358,Farmers are rights,farmer be right,0
3360,Spineless modi now this will boost confidence...,spineless modi now this will boost confidence ...,0
3362,These three laws are very beneficial to farmer...,these three law be very beneficial to farmer o...,1
3363,Farmers are not terrorist,farmer be not terrorist,0
3419,"We stand with Indian farmers, down with all o...",we stand with indian farmer down with all oppr...,0


### Weak Supervision